# Feature Engineering
Load the `adult` dataset. Preprocess the dataset by removing missing values, encoding categorical attributes, normalizing/scaling the features, and reducing the dimensionality of the dataset. Split the dataset into training and test sets. Train and test a support vector machine model using scikit-learn. Evaluate the impact of the feature engineering step on the effectiveness and efficiency of the model.

## Loading Modules

In [ ]:
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.svm
import sklearn.metrics
import sklearn.preprocessing
import sklearn.decomposition
import plotly.express as px

## Loading the Dataset

In [ ]:
df = pd.read_csv("adult.csv")
print(df.shape)
df.head()

(17923, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


## Removing Missing Values

In [ ]:
# print(df.columns)
# df["workclass"].value_counts()
df = df.replace(' ?', np.NaN) # None
df = df.dropna()   # fillna(0)
df.shape

(16607, 15)

## Splitting the Data into Training and Test Sets

In [ ]:
x = df.drop(["target"], axis=1)
y = df["target"]
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

print("df:", df.shape)
print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

df: (16607, 15)
x_train: (12455, 14)
x_test: (4152, 14)
y_train: (12455,)
y_test: (4152,)


## One-Hot Encoding

In [ ]:
# Building the one-hot encoder model
enc = sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore")
enc.fit(x_train)

# Encoding the categorical attriutes of training data
x_train = enc.transform(x_train)

# Encoding the categorical attriutes of test data
x_test = enc.transform(x_test)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)

x_train: (12455, 10610)
x_test: (4152, 10610)


## Standardization

In [ ]:
# Building a standardization model
scaler = sklearn.preprocessing.StandardScaler(with_mean=False)
scaler.fit(x_train)

# scaling the training features
x_train = scaler.transform(x_train)

# Reducing the number of test features
x_test = scaler.transform(x_test)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)

x_train: (12455, 10610)
x_test: (4152, 10610)


## Feature Reduction

In [ ]:
# Building a PCA model
pca = sklearn.decomposition.PCA(n_components=1000)
pca.fit(x_train.toarray())

# Reducing the number of training features
x_train = pca.transform(x_train.toarray())

# Reducing the number of test features
x_test = pca.transform(x_test.toarray())

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)

x_train: (12455, 1000)
x_test: (4152, 1000)


## Training a Model

In [ ]:
model = sklearn.svm.SVC()
model.fit(x_train, y_train);

## Test the Model

In [ ]:
y_predicted = model.predict(x_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_predicted)
accuracy

0.8542870905587668

## Hyperparameter Tuning

In [ ]:
n_component_list = [10, 1000, 10000]
result_df = pd.DataFrame(columns=["N", "Accuracy"])

x = df.drop(["target"], axis=1)
y = df["target"]
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

# Building the one-hot encoder model
enc = sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore")
enc.fit(x_train)

# Encoding the categorical attriutes of training data
x_train = enc.transform(x_train).toarray()

# Encoding the categorical attriutes of test data
x_test = enc.transform(x_test).toarray()

for n in n_component_list:
  
  # Building a PCA model
  pca = sklearn.decomposition.PCA(n_components=n)
  pca.fit(x_train)

  # Reducing the number of training features
  x_train_ = pca.transform(x_train)

  # Reducing the number of test features
  x_test_ = pca.transform(x_test)

  # Training a model
  model = sklearn.svm.SVC()
  model.fit(x_train_, y_train);

  # Testing the Model
  y_predicted = model.predict(x_test_)
  accuracy = sklearn.metrics.accuracy_score(y_test, y_predicted)

  result_df = result_df.append({"N": n, "Accuracy": accuracy}, ignore_index=True)

result_df

,N,Accuracy
0,10.0,0.823699
1,1000.0,0.857177
2,10000.0,0.858382
